In [ ]:
%pip install torch clickhouse_connect scipy transformers accelerate

In [ ]:
import clickhouse_connect, torch
from scipy.spatial import distance
from transformers import AutoModel, AutoTokenizer, Conversation, pipeline